In [ ]:
#ESERCIZIO1: ESTRARRE LA TABELLA dimproduct dal database AdventureWorks
''' sulla colonna DealerPrice, utilizzando il metodo .round(), arrotondare i valori alle due cifre decimali, e poi al valore intero più vicino
    utilizzando il metodo .clip(), fare in modo che i valori siano compresi tra un minimo di 0 e un massimo di 1000'''

import os
import dotenv 
import sqlalchemy
import pandas as pd

dotenv.load_dotenv(dotenv_path="adv_sql.env", override=True,) #carica il file env

username = os.getenv("username") #credenziali
password = os.getenv("password")
host = os.getenv("host")
dbname = os.getenv("dbname")

connection_string = f"mysql+pymysql://{username}:{password}@{host}/{dbname}" #stringa di connessione
db_engine = sqlalchemy.create_engine(connection_string) #connessione al db

query = "SELECT * FROM dimproduct"
df = pd.read_sql(query, db_engine)

df["DealerPrice"] = df["DealerPrice"].round(2) #arrotondo alle due cifre decimali
df["DealerPrice"] = df["DealerPrice"].round(0) #arrotondo all'intero più vicino
df["DealerPrice"] = df["DealerPrice"].clip(lower=0, upper=1000) #setto i valori tra 0 e 1000

df["DealerPrice"].sample(5)

In [ ]:
#ESERCIZIO2: creare un DataFrame sintetico, che contiene i guadagni mensili di diverse annate
''' calcolare la somma cumulativa dei guadagni utilizzando il metodo .cumsum()
    come sopra, ma raggruppato per ogni anno usando prima un .groupby() '''

import numpy as np
import pandas as pd

years = 5 
guadagni = pd.DataFrame({"Mese": list("GFMAMGLASOND"*years), 
                         "Anno": np.repeat(list(range(years)), 12), 
                         "Valore": np.random.randint(800, 5000, 12*years)})

#creo colonna vendite cumulative e inserisco la cumsum
guadagni.loc[:, "Vendite Cumulaive"] = guadagni.loc[:, "Valore"].cumsum() 

anni = guadagni.groupby("Anno")["Valore"].cumsum()
anni


In [ ]:
#ESERCIZIO3: ESTRARRE LA TABELLA dimcustomer dal database AdventureWorks
''' trasformare  i nomi dei clienti in modo che abbiano solo lettere minuscole, e i cognomi in modo che abbiano solo lettere maiuscole
    sulla colonna EmailAddress, utilizzando il metodo .str.split(), estrarre nome utente e dominio
    sulla colonna Phone, estrarre ogni parte del numero (ad es. da "1 (11) 500 555-0162" a ["1", "(11)", "500", "555-0162"])
    utilizzando il metodo .str.contains(), estrarre tutti gli indirizzi e-mail che contengono il numero "21" '''

import os
import dotenv 
import sqlalchemy
import pandas as pd

dotenv.load_dotenv(dotenv_path="adv_sql.env", override=True,) #carica il file env

username = os.getenv("username") #credenziali
password = os.getenv("password")
host = os.getenv("host")
dbname = os.getenv("dbname")

connection_string = f"mysql+pymysql://{username}:{password}@{host}/{dbname}" #stringa di connessione
db_engine = sqlalchemy.create_engine(connection_string) #connessione al db

query = "SELECT * FROM dimcustomer"
df = pd.read_sql(query, db_engine)

df["FirstName"] = df["FirstName"].str.lower() #nomi minuscoli
df["LastName"] = df["LastName"].str.upper() #cognomi maiuscoli

#df["EmailAddress"] = df["EmailAddress"].str.split(pat="@") #divido utente e dominio separati da @

df["Phone"] = df["Phone"].str.split() #divido le cifre separate da whitespace

num_21 = df.loc[df["EmailAddress"].str.contains("21", regex=False)]  #cerco le email con 21 


In [ ]:
#ESERCIZIO3: ESTRARRE LA TABELLA dimcustomer dal database AdventureWorks
''' estrarre tutti gli indirizzi e-mail che contengono il numero "20" oppure il numero "10" 
    calcolare la lunghezza di ogni indirizzo e-mail ed estrarre i cinque più lunghi e i cinque più corti 
    modificare il dominio degli indirizzi e-mail da "adventure-works.com" a "aw-db.com" mediante il metodo .str.replace() 
    dalla colonna AddressLine1 estrarre tutti gli indirizzi che contengono la sottostringa "Street" '''

import os
import dotenv 
import sqlalchemy
import pandas as pd

dotenv.load_dotenv(dotenv_path="adv_sql.env", override=True,) #carica il file env

username = os.getenv("username") #credenziali
password = os.getenv("password")
host = os.getenv("host")
dbname = os.getenv("dbname")

connection_string = f"mysql+pymysql://{username}:{password}@{host}/{dbname}" #stringa di connessione
db_engine = sqlalchemy.create_engine(connection_string) #connessione al db

query = "SELECT * FROM dimcustomer"
df = pd.read_sql(query, db_engine)

num_20 = df["EmailAddress"].str.contains(("20"), regex=False) #email con 20
num_10 = df["EmailAddress"].str.contains(("10"), regex=False) #email con 10
num_20_10 = df.loc[num_20 | num_10] #filtro email

emails = df["EmailAddress"]
lista = []

#per ogni email conto la lunghezza della email, si inserisce tutto in una lista e stampo i primi 5 e ultimi 5
for email in emails: 
    lung = len(email)
    lista.append(lung)

lista.sort(reverse=False)
#print(lista[0:5]) #primi 5
#print(lista[-5:]) #ultimi 5

#con .str.len()
emails = df["EmailAddress"]
lunghezze = emails.str.len()
sort_lung = lunghezze.sort_values()
primi_5 = sort_lung.head(5)
ultimi_5 = sort_lung.tail(5)

#print(primi_5, ultimi_5)

df["EmailAddress"] = df["EmailAddress"].str.replace("adventure-works.com", "aw-db.con") #sostituisco dominio

street = df.loc[df["AddressLine1"].str.contains("Street", regex=False)]
street


In [ ]:
#ESERCIZIO4: CARICARE IL DATASET facebook.csv che contiene dei post con data di pubblicazione, tipo (foto, video, …) e numero di reactions raccolte:
''' con la funzione pd.to_datetime() convertire la colonna status_published in formato Timestamp
    utilizzando gli attributi .dt.year , .dt.month , .dt.day , .dt.dayofweek , .dt.dayofyear, 
    ottenere informazioni specifiche sulle date delle transazioni, come l'anno, il mese, il giorno della settimana, il giorno dell'anno
    estrarre solo i post relativi al 2012
    estrarre solo i post relativi a maggio 2018
    confrontare il numero di post pubblicati nei weekend rispetto al numero di post pubblicati nel resto della settimana
    trovare il primo e ultimo post pubblicati in ogni anno
    quanti tipi di post ci sono? e quanti per ogni tipo? '''

import pandas as pd
url = "Dataset/facebook.csv"
df = pd.read_csv(url)

df["status_published"] = pd.to_datetime(df.loc[:, "status_published"]) #trasformo in timestamp

#anno, mese, giorno, giorno della settimana e giorno dell'anno
df["year"] = df.loc[:, "status_published"].dt.year
df["month"] = df.loc[:, "status_published"].dt.month
df["day"] = df.loc[:, "status_published"].dt.day
df["weekday"] = df.loc[:, "status_published"].dt.dayofweek
df["yearday"] = df.loc[:, "status_published"].dt.dayofyear

filtro_2012 = df.loc[:, "year"] == 2012 #filtro che prende i post dell'anno 2012
post_2012 = df.loc[filtro_2012]

filtro_05_2018 = (df.loc[:, "year"] == 2018) & (df.loc[:, "month"]== 5 )
post_05_2018 = df.loc[filtro_05_2018]

num_post = df.groupby("weekday").size() #numero di post pubblicati nei giorni della settimana

first = df.groupby("year")["status_type"].first() #primo post per ogni anno
last = df.groupby("year")["status_type"].last() #ultimo post per ogni anno

df.groupby("status_type").size() #tipi di post e quanti per ogni tipo

In [ ]:
#ESERCIZIO5: CARICARE IL DATASET pokemon.csv 
''' Tramite i metodi .isnull() e .sum() controlliamo se ci sono valori nulli nel dataset e contiamo quanti valori nulli ci sono in ogni colonna 
    ci sono valori nulli? se sì, avrebbe senso cercare di riempirli? 
    eliminare le righe che contengono valori nulli '''

import pandas as pd
url = "Dataset/pokemon.csv"
df = pd.read_csv(url)

df.isnull().sum() #valori nulli, avrebbe senso riempirli solo per mantenere tutte le righe
df.dropna() #elimino righe che hanno valori null, così si perdono dati però

In [ ]:
#ESERCIZIO6: CARICARE IL DATASET automobile.csv
''' ci sono valori nulli? dove? quanti?
    quali righe hanno un valore nullo nella colonna num-of-doors?
    esaminando i dati nel dataset, cercare una logica per sostituire i valori nulli nella colonna num-of-doors '''

import pandas as pd
url = "Dataset/automobile.csv"
df = pd.read_csv(url)

df.isnull().sum()  #37 nulli in normalized-losses e 2 in num-of-doors
df["num-of-doors"] = df["num-of-doors"].fillna(2) #riempio con il minimo di porte 



In [ ]:
#ESERCIZIO7: dato il seguente dataframe che raccoglie le misurazioni di un sensore che misura la temperatura atmosferica giornaliera
''' il sensore a volte non funziona, dunque alcuni dati sono mancanti: quale sarebbe la migliore strategia per gestirli? 
    nella directory dei beginner_datasets, quali sono i dataset che contengono dati nulli? (vedere slide)
    usare un ciclo for per esaminare tutti i nomi dei file
    selezionare solo i nomi di file con estensione .csv (costrutto if)
    leggere di volta in volta il file in esame, e caricarlo in un DataFrame con la funzione .read_csv()
    utilizzare il metodo .isna() per trovare la maschera booleana dei dati nulli
    contare i dati nulli, utilizzando .sum(); potremmo doverlo utilizzare più di una volta
    stampare, o memorizzare in una list, solo i nomi dei file che contengono dati nulli'''

import os
import numpy as np
import pandas as pd 


temp = pd.DataFrame({"Giorno": [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12], 
                     "Temperature": [18, 19, 18, np.nan, 21, 20, 20, np.nan, 21, 23, np.nan, 23, 24]})

#dato che il sensore a volte non funziona, sarebbe meglio un'interpolazione 
temp.interpolate()

datasets = os.listdir("Datasets_EPICODE/beginner_datasets") #lista nomi in directory
lista = [] #inizializzazione per la lista finale
for dataset in datasets: 
    if dataset[-4:] == ".csv": #prendo i dataset che terminano con .csv
        df = pd.read_csv(f"Datasets_EPICODE/beginner_datasets/{dataset}") #leggo i file csv
        nulli = df.isna().sum().sum() #calcolo nulli nel df 
        if nulli > 1: #se sono presenti nulli, si aggiunge il df nella lista
            lista.append(dataset)
print(lista)



['automobile.csv', 'elections.csv', 'france.csv', 'hepatitis.csv', 'house.csv', 'income.csv', 'mice.csv', 'nba.csv', 'pokemon.csv', 'population.csv', 'seeds.csv', 'traffic.csv', 'wikipedia.csv']
